In [ ]:
#!wget -O multi_episode_dataset.npz https://www.dropbox.com/scl/fi/plgy3w7i4r7agd43b83eh/multi_episode_dataset.npz?rlkey=v6vw2i759ovp1e0laagq8d9th&dl=1

In [ ]:
! ls -la

: 

In [3]:
#! pip install -U imitation stable-baselines3 --quiet


In [4]:
#! pip install imitation --quiet

Librerias


In [1]:
#
import numpy as np
from imitation.data.types import TrajectoryWithRew
from imitation.data.rollout import flatten_trajectories
from imitation.algorithms.bc import BC
from stable_baselines3 import PPO

2024-12-08 21:47:00.905107: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-08 21:47:00.916701: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-08 21:47:00.932187: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-08 21:47:00.936430: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-08 21:47:00.948412: I tensorflow/core/platform/cpu_feature_guar

In [2]:

# 1) Cargar el archivo .npz de trayectorias
data = np.load("multi_episode_dataset.npz", allow_pickle=True)
dataset_episodios = data["trajectories"]


In [7]:
# print(dataset_episodios)


In [3]:
# Ajustar las trayectorias para asegurarse de que hay una observación extra
trajectories = []
for episode in dataset_episodios:
    observations = episode["obs"]
    actions = episode["acts"]

    # Convertir a lista para añadir una observación extra si es necesario
    if len(observations) == len(actions):
        observations = np.append(observations, [observations[-1]], axis=0)

    # Crear el objeto `TrajectoryWithRew`
    trajectory = TrajectoryWithRew(
        obs=observations,
        acts=actions,
        infos=episode["infos"],
        rews=episode["rews"],
        terminal=episode["dones"]
    )
    trajectories.append(trajectory)


In [4]:
# 2)  Crear una lista de episodios en el formato esperado


# 3) Crear las transiciones a partir de las trayectorias
transitions = flatten_trajectories(trajectories)


In [5]:
from stable_baselines3.common.logger import configure, Logger
import os
from imitation.algorithms import bc


# Crear un directorio para guardar los logs
log_dir = "./logs/bc_logs_20241208"
os.makedirs(log_dir, exist_ok=True)


# Configurar el logger globalmente para TensorBoard y CSV
logger = configure(log_dir, ["tensorboard", "csv"])



In [ ]:
import gymnasium as gym

o_s = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(10,), dtype=np.float32)
a_s = gym.spaces.Discrete(6)


# Create a random number generator
rng = np.random.default_rng(0)

# Inicializar BC con el conjunto de transiciones
bc_trainer = BC(observation_space=o_s,action_space=a_s, demonstrations=transitions, rng=rng, custom_logger=logger)

#bc_trainer.logger = logger


In [8]:

# Entrenar la política
bc_trainer.train(n_epochs=100)  # Ajusta según tus necesidades


0batch [00:00, ?batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 0        |
|    ent_loss       | -0.00179 |
|    entropy        | 1.79     |
|    epoch          | 0        |
|    l2_loss        | 0        |
|    l2_norm        | 84.5     |
|    loss           | 1.79     |
|    neglogp        | 1.79     |
|    prob_true_act  | 0.167    |
|    samples_so_far | 32       |
--------------------------------


491batch [00:01, 292.13batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 500      |
|    ent_loss       | -0.00107 |
|    entropy        | 1.07     |
|    epoch          | 0        |
|    l2_loss        | 0        |
|    l2_norm        | 112      |
|    loss           | 0.864    |
|    neglogp        | 0.865    |
|    prob_true_act  | 0.48     |
|    samples_so_far | 16032    |
--------------------------------


987batch [00:03, 303.03batch/s]

...

64479batch [04:04, 293.35batch/s]

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 64500     |
|    ent_loss       | -3.79e-05 |
|    entropy        | 0.0379    |
|    epoch          | 98        |
|    l2_loss        | 0         |
|    l2_norm        | 4.7e+03   |
|    loss           | 0.00872   |
|    neglogp        | 0.00875   |
|    prob_true_act  | 0.991     |
|    samples_so_far | 2064032   |
---------------------------------


64966batch [04:06, 348.91batch/s]

---------------------------------
| batch_size        | 32        |
| bc/               |           |
|    batch          | 65000     |
|    ent_loss       | -8.34e-06 |
|    entropy        | 0.00834   |
|    epoch          | 99        |
|    l2_loss        | 0         |
|    l2_norm        | 4.72e+03  |
|    loss           | 0.0011    |
|    neglogp        | 0.0011    |
|    prob_true_act  | 0.999     |
|    samples_so_far | 2080032   |
---------------------------------


65500batch [04:08, 263.83batch/s]


In [9]:

# Formato de la política entrenada
bc_trainer.policy

FeedForward32Policy(
  (features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (pi_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (vf_features_extractor): FlattenExtractor(
    (flatten): Flatten(start_dim=1, end_dim=-1)
  )
  (mlp_extractor): MlpExtractor(
    (policy_net): Sequential(
      (0): Linear(in_features=10, out_features=32, bias=True)
      (1): Tanh()
      (2): Linear(in_features=32, out_features=32, bias=True)
      (3): Tanh()
    )
    (value_net): Sequential(
      (0): Linear(in_features=10, out_features=32, bias=True)
      (1): Tanh()
      (2): Linear(in_features=32, out_features=32, bias=True)
      (3): Tanh()
    )
  )
  (action_net): Linear(in_features=32, out_features=6, bias=True)
  (value_net): Linear(in_features=32, out_features=1, bias=True)
)

In [10]:
bc_trainer.policy.state_dict()

OrderedDict([('mlp_extractor.policy_net.0.weight',
              tensor([[-2.1811e+00,  3.8088e-02, -1.4004e+01,  4.6421e-01, -3.3287e-01,
                        4.3926e-01, -3.1623e-01,  4.1892e-01,  1.5573e-01,  5.1046e-01],
                      [-7.4424e+00,  8.6476e-01, -1.6369e+01, -3.1812e-01,  5.6170e-01,
                       -2.6022e-01, -1.6200e-02,  9.8911e-01, -7.3977e-01,  1.1996e+00],
                      [ 6.7713e+00, -3.1825e+00, -1.4264e+01,  3.8961e-01, -2.0860e-01,
                        7.1031e-02,  5.9025e-01, -1.1148e-01, -2.0878e+00,  2.9099e+00],
                      [-1.4616e+01,  1.0692e+01, -8.1402e+00, -2.7050e-01,  9.7047e-02,
                       -6.9059e-01,  2.7308e-01,  1.9857e-01, -7.4917e-01,  5.6908e-01],
                      [-2.5695e-03,  3.7408e+00,  1.1438e+01, -6.3350e-01,  1.5189e-01,
                       -2.4289e-01,  4.4575e-01, -1.7976e-01, -5.4384e-01, -9.5664e-02],
                      [-9.6635e+00, -9.6948e-01, -2.4255e+00, -9

In [12]:
import torch

# Definir la ruta para guardar la política entrenada
policy_path = "./logs/bc_logs_20241208/bc_policy"

# Guardar la política entrenada del modelo BC
torch.save(bc_trainer.policy.state_dict(), policy_path)

Voy a probar que puedo cargar la politica sin problemas de nuevo


In [13]:

import torch
from imitation.algorithms import bc
# Inicializar BC con el conjunto de transiciones
bc_model = BC(observation_space=o_s,action_space=a_s, demonstrations=None, rng=rng)

# Cargar los pesos en la política del modelo BC
bc_model.policy.load_state_dict(torch.load(policy_path))

<All keys matched successfully>

In [ ]:
# CODIGO EJEMPLO 
# Evaluación del modelo BC cargado
n_episodes = 10
all_rewards = []
for episode in range(n_episodes):
    obs = env.reset()
    done = False
    total_reward = 0

    while not done:
        # Obtener la acción del modelo BC
        action = bc_model.policy.predict(obs, deterministic=True)
        
        # Ejecutar la acción en el entorno
        obs, reward, done, info = env.step(action)
        
        # Acumular la recompensa
        total_reward += reward
        
        # Renderizar el entorno (opcional)
        env.render()

    all_rewards.append(total_reward)
    print(f"Episodio {episode + 1}: Recompensa total = {total_reward}")

# Calcular y mostrar métricas de evaluación
mean_reward = np.mean(all_rewards)
std_reward = np.std(all_rewards)
print(f"Recompensa media sobre {n_episodes} episodios: {mean_reward}")
print(f"Desviación estándar de la recompensa: {std_reward}")

# Cerrar el entorno
env.close()
